In [ ]:
!pip install /kaggle/input/icevision-052/fastcore-1.3.2-py3-none-any.whl /kaggle/input/icevision-052/loguru-0.5.3-py3-none-any.whl /kaggle/input/icevision-052/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl /kaggle/input/icevision-052/imagesize-1.2.0-py2.py3-none-any.whl /kaggle/input/icevision-052/timm-0.4.5-py3-none-any.whl /kaggle/input/icevision-052/omegaconf-2.0.6-py3-none-any.whl /kaggle/input/icevision-052/effdet-0.2.1-py3-none-any.whl /kaggle/input/icevision-052/icevision-0.5.2-py3-none-any.whl

In [ ]:
!pip uninstall fastai -y

[Train notebook](https://www.kaggle.com/nikitautin/35th-place-efficientdet-train)

In [ ]:
import numpy as np
import pandas as pd
import os
from icevision.all import *
from tqdm.contrib.concurrent import process_map
from functools import partial
from effdet import DetBenchPredict, unwrap_bench

In [ ]:
FRAME_RANGE = 4
VALID_PERCENT = 0.2
SIZE = (512, 512)
CLASSES_NUM = 2
IMPACT_CLASS = 2

IOU_THR = 0.35
FILTER_OFFSET = 50

In [ ]:
path = Path("/kaggle/input/nfl-impact-detection")
test_video_path = path/'test'

In [ ]:
def make_images(video_name, video_dir, video_labels, out_dir, only_with_impact=True, impact_cls=IMPACT_CLASS):
    vidcap = cv2.VideoCapture(str(video_dir/video_name))
    frame = 0
    while True:
        read, img = vidcap.read()
        if not read:
            break
        frame += 1
        if only_with_impact:
            query_str = 'video == @video_name and frame == @frame and impact == @impact_cls'
            boxes = video_labels.query(query_str)
            if len(boxes) == 0:
                continue
        image_path = f'{out_dir}/{video_name}'.replace('.mp4', f'_{frame}.png')
        _ = cv2.imwrite(image_path, img)

In [ ]:
test_images_path = Path('/kaggle/working/test_images')
test_images_path.mkdir()

In [ ]:
make_images_part = partial(make_images, video_dir=test_video_path, video_labels=None,
                           out_dir=test_images_path, only_with_impact=False)
process_map(make_images_part, os.listdir(test_video_path), max_workers=2);

In [ ]:
infer_tfms = tfms.A.Adapter([tfms.A.Resize(*SIZE), tfms.A.Normalize()])

In [ ]:
class InferParser(parsers.Parser, parsers.FilepathMixin, parsers.SizeMixin):
    def __init__(self, path):
        self.images = get_image_files(path)

    def __iter__(self):
        yield from self.images

    def __len__(self):
        return len(self.images)

    def imageid(self, o) -> Hashable:
        return o.stem

    def filepath(self, o) -> Union[str, Path]:
        return o

    def image_width_height(self, o) -> Tuple[int, int]:
        return get_image_size(self.filepath(o))

In [ ]:
parser = InferParser("/kaggle/working/test_images")
splitter = SingleSplitSplitter()

In [ ]:
infer_rs = parser.parse(data_splitter=splitter, autofix=False)[0]

In [ ]:
infer_ds = Dataset(infer_rs, infer_tfms)

In [ ]:
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/nfl-models/tf_efficientnet_b5_ra-9a3e5369.pth /root/.cache/torch/hub/checkpoints/

In [ ]:
model = efficientdet.model(model_name="tf_efficientdet_d5", num_classes=CLASSES_NUM, img_size=SIZE, pretrained=False)

In [ ]:
state_dict = torch.load('/kaggle/input/nfl-models/efdb5_512_mixup_2cl_5-10ep.pth', map_location='cuda:0')

In [ ]:
model.load_state_dict(state_dict)

In [ ]:
model.eval();

In [ ]:
infer_dl = efficientdet.infer_dl(infer_ds, batch_size=64)

In [ ]:
box_list = []
score_list = []
detection_threshold = 0.4

with torch.no_grad():
    device = torch.device('cuda:0')
    bench = DetBenchPredict(unwrap_bench(model))
    bench = bench.eval().to(device)

    for batch, _ in tqdm(infer_dl):
        imgs, img_info = batch
        imgs = imgs.to(device)
        img_info = {k: v.to(device) for k, v in img_info.items()}

        raw_preds = bench(x=imgs, img_info=img_info)
        dets = raw_preds.detach().cpu().numpy()

        for det in dets:
            boxes = det[:, :4]
            scores = det[:, 4]
            labels = det[:, 5]
            indexes = np.where((scores > detection_threshold)
                               & (labels == IMPACT_CLASS))[0]
            box_list.append(boxes[indexes])
            score_list.append(scores[indexes])

Convert bounding boxes from xyxy to xywh and to origin image size:

In [ ]:
result_image_ids = []
results_boxes = []
results_scores = []
for i in range(len(box_list)):
    boxes = box_list[i]
    scores = score_list[i]
    image_id = infer_rs[i].filepath.name
    boxes[:, 0] = (boxes[:, 0] * 1280 / SIZE[1])
    boxes[:, 1] = (boxes[:, 1] * 720 / SIZE[0])
    boxes[:, 2] = (boxes[:, 2] * 1280 / SIZE[1])
    boxes[:, 3] = (boxes[:, 3] * 720 / SIZE[0])
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    boxes = boxes.astype(np.int32)
    boxes[:, 0] = boxes[:, 0].clip(min=0, max=1280-1)
    boxes[:, 2] = boxes[:, 2].clip(min=0, max=1280-1)
    boxes[:, 1] = boxes[:, 1].clip(min=0, max=720-1)
    boxes[:, 3] = boxes[:, 3].clip(min=0, max=720-1)
    result_image_ids += [image_id] * len(boxes)
    results_boxes.append(boxes)
    results_scores.append(scores)
    
results_boxes = np.concatenate(results_boxes)
results_scores = np.concatenate(results_scores)

In [ ]:
len(results_boxes)

In [ ]:
box_df = pd.DataFrame(results_boxes, columns=['left', 'top', 'width', 'height'])
test_df = pd.DataFrame({'scores':results_scores, 'image_name':result_image_ids})
test_df = pd.concat([test_df, box_df], axis=1)

In [ ]:
test_df['gameKey'] = test_df.image_name.str.split('_').str[0].astype(int)
test_df['playID'] = test_df.image_name.str.split('_').str[1].astype(int)
test_df['view'] = test_df.image_name.str.split('_').str[2]
test_df['frame'] = test_df.image_name.str.split('_').str[3].str.replace('.png','').astype(int)
test_df['video'] = test_df.image_name.str.rsplit('_',1).str[0] + '.mp4'
test_df = test_df[["gameKey","playID","view","video","frame","left","width","top","height","scores"]]
test_df.head()

In [ ]:
def iou(bbox1, bbox2):
    bbox1 = [float(x) for x in bbox1]
    bbox2 = [float(x) for x in bbox2]

    (x0_1, y0_1, x1_1, y1_1) = bbox1
    (x0_2, y0_2, x1_2, y1_2) = bbox2
    x1_1 += x0_1
    y1_1 += y0_1
    x1_2 += x0_2
    y1_2 += y0_2

    # get the overlap rectangle
    overlap_x0 = max(x0_1, x0_2)
    overlap_y0 = max(y0_1, y0_2)
    overlap_x1 = min(x1_1, x1_2)
    overlap_y1 = min(y1_1, y1_2)

    # check if there is an overlap
    if overlap_x1 - overlap_x0 <= 0 or overlap_y1 - overlap_y0 <= 0:
            return 0

    # if yes, calculate the ratio of the overlap to each ROI size and the unified size
    size_1 = (x1_1 - x0_1) * (y1_1 - y0_1)
    size_2 = (x1_2 - x0_2) * (y1_2 - y0_2)
    size_intersection = (overlap_x1 - overlap_x0) * (overlap_y1 - overlap_y0)
    size_union = size_1 + size_2 - size_intersection

    return size_intersection / size_union

NMS with frame range to filter false impact:

In [ ]:
keep_idx = []
for keys in test_df.groupby(['gameKey', 'playID']).size().to_dict().keys():
    for view in ["Endzone", "Sideline"]:
        tmp_df = test_df.query('gameKey == @keys[0] and playID == @keys[1] and view == @view').copy()
        while (len(tmp_df) > 0):
            boxes = tmp_df[["left", "top", "width", "height"]].to_numpy()
            max_box_idx = tmp_df["scores"].idxmax()
            max_box = tmp_df["scores"].argmax()
            ious = np.array(list(map(partial(iou, boxes[max_box]), boxes)))
            frame = tmp_df.loc[max_box_idx, "frame"]
            m = (ious > IOU_THR) & (tmp_df["frame"].to_numpy() <= frame + FILTER_OFFSET) & (tmp_df["frame"].to_numpy() >= frame - FILTER_OFFSET)
            keep_idx.append(max_box_idx)
            tmp_df.drop(tmp_df[m].index, inplace=True)
test_df = test_df[test_df.index.isin(keep_idx)].copy()

In [ ]:
len(test_df)

In [ ]:
test_df.drop(columns=['scores'], inplace=True)
test_df.head(3)

In [ ]:
import nflimpact

env = nflimpact.make_env()
env.predict(test_df)

In [ ]:
!rm -rf /kaggle/working/test_images/*